## Assessment assignments answered

This will be the notebook where everything relating to the assessment exercices happens, coupled with a small explanation of how and why for each.

### 1. Acquiring the data, and making it useful

we want a ranking list of countried by number of airports, then the top and bottom 10, along with their respective number of airports, as well as the width and length of the longest runway for each country, and which airport that runway belongs to.

#### Importing the csv files, and answering parts 1 and 2

In [1]:
import pandas as pd

airports  = pd.read_csv("airports.csv")
countries = pd.read_csv("countries.csv")
runways   = pd.read_csv("runways.csv")

# We can take the list of airports and count them per country, then sorting the result
num_airports = airports.groupby("iso_country").size().sort_values(ascending=False).reset_index(name="counts")

# To easily read the country names, we pull the names from the second file, and make the translation
country_ids = countries.loc[:, "code" : "name"].set_index("code")
num_airports = num_airports.set_index("iso_country").merge(country_ids, left_on="iso_country", right_on="code").set_index("name")
num_airports.rename(columns={"counts" : "airports"}, inplace=True)

assert countries.shape[0] >= num_airports.shape[0]

print("The 10 countries with the most airports are:")
print(num_airports.head(10))

# We're showing the 15 countries with the least airports, to show that there are 14 with exactly 1, so those 14 all share the last spot
print("The 10 countries with the least airports are:")
print(num_airports.tail(15))

The 10 countries with the most airports are:
                airports
name                    
United States      25170
Brazil              5546
Canada              2829
Australia           2038
Mexico              1779
Japan               1761
South Korea         1378
Russia              1306
United Kingdom      1289
Germany              967
The 10 countries with the least airports are:
                                airports
name                                    
Saint Pierre and Miquelon              2
Gambia                                 1
Mayotte                                1
Gibraltar                              1
Vatican City                           1
British Indian Ocean Territory         1
Jersey                                 1
Niue                                   1
Nauru                                  1
Saint Barthélemy                       1
Christmas Island                       1
Curaçao                                1
Sint Maarten                       

#### Answering the runway question

The list of longest runway width and length per country is a bit more tricky, but still very doable; every runway belongs to an airport, situated in a country, so we need to add the country each runway is in to the dataframe, so we can group by country.

In [2]:
# First we link all airport ids to the country the airport is in, then do the same with all runways
airport_countries = airports.loc[:, : "iso_country"].merge(country_ids, left_on="iso_country", right_on="code").set_index("id")
airport_countries = airport_countries.loc[:,"name_x" :].rename(columns={"name_x" : "airport", "name_y" : "country"})
airport_countries.drop(columns=["latitude_deg", "longitude_deg", "elevation_ft", "continent", "iso_country"], inplace=True)
longest_runways = airport_countries.merge(runways, left_on="id", right_on="airport_ref")
del airport_countries

# having linked the runways to their respective airports and countries, it's a matter of grouping the countries by longest runway
longest_runways = longest_runways[longest_runways.groupby(["country"])["length_ft"].transform(max) == longest_runways["length_ft"]]
longest_runways = longest_runways.set_index("country").loc[:, : "width_ft"].drop(columns=["id", "airport_ref", "airport_ident"])
longest_runways = longest_runways.reset_index().drop_duplicates(["country"]).sort_values("country").set_index("country")

assert countries.shape[0] >= longest_runways.shape[0]

for row in longest_runways.itertuples(True):
    print(f"The longest runway in {row[0]} is {row.length_ft}ft long, {row.width_ft} wide, and belongs to the airport named {row.airport}")

The longest runway in Afghanistan is 11820.0ft long, 151.0 wide, and belongs to the airport named Bagram Airfield
The longest runway in Albania is 9318.0ft long, 220.0 wide, and belongs to the airport named Kuçovë Air Base
The longest runway in Algeria is 12737.0ft long, 246.0 wide, and belongs to the airport named Tinfouchy Airport
The longest runway in American Samoa is 10000.0ft long, 150.0 wide, and belongs to the airport named Pago Pago International Airport
The longest runway in Angola is 12190.0ft long, 148.0 wide, and belongs to the airport named Quatro de Fevereiro Airport
The longest runway in Anguilla is 5462.0ft long, 98.0 wide, and belongs to the airport named Clayton J Lloyd International Airport
The longest runway in Antarctica is 12000.0ft long, 200.0 wide, and belongs to the airport named South Pole Station Airport
The longest runway in Antigua and Barbuda is 9003.0ft long, 148.0 wide, and belongs to the airport named V.C. Bird International Airport
The longest runway 

### 2. Uploading to cloud storage

Now that we have these answers, we want to upload the dataframes that give us these answers to a cloud block-storage service. We're given the credentials, so we use those, and upload the dataframes after writing them to csv files first.

In [3]:
from azure.storage.blob import *

acc_name  = "sacodeassessment"
acc_SAS   = "se=2022-03-10T15%3A46Z&sp=rcwd&spr=https&sv=2018-11-09&sr=c&sig=Wf6AQ%2BoLm3mtRq%2BHTvvGt7BzBmUOqz/4QPzTUezcUZo%3D"
container = "results"
blob_path = "/ingest-airports-20220216-robert"

blob = BlobClient.from_blob_url(f"https://{acc_name}.blob.core.windows.net/" + container + blob_path, credential=acc_SAS)

if blob.exists():
    blob.delete_blob("include")
answers = num_airports.join(longest_runways).rename_axis("country")
blob.upload_blob(answers.to_csv())


{'etag': '"0x8D9F86F8B30634B"',
 'last_modified': datetime.datetime(2022, 2, 25, 15, 0, 20, tzinfo=datetime.timezone.utc),
 'content_md5': bytearray(b'\xe0\xaa\x8a$\xdcc\x1a\xf8\x9e\xc0EO\xa3\xbb4&'),
 'client_request_id': 'a6cfd0c6-964b-11ec-b225-ec63d7c84d34',
 'request_id': '274ddfab-f01e-000e-8058-2a1f7f000000',
 'version': '2020-10-02',
 'version_id': None,
 'date': datetime.datetime(2022, 2, 25, 15, 0, 20, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}

### Navigating eCommerce RDBMS

We aim to evaluate the total spending of all people who've bought jazz music against people who haven't. This is spending per person, so we need to accumulate the totals for every buyer, then seperate then into jazz and non-jazz, and average those values, comparing those final two values. All info we need for this is in an sql database, with the given credentials.

In [4]:
password = input("Please input the password:")

To keep the password private, I'm asking for the password as input when running the previous block of code, so we can use the connection without constantly asking for the password going forward.

In [5]:
from sqlalchemy import create_engine
import psycopg2 as psy

# Connecting to the PostgreSQL Server v13, by making the sqlalchemy engine
dialect  = "postgresql"
driver   = "psycopg2"
hostname = "code-assessment-001.westeurope.cloudapp.azure.com"
port     = 5432
database = "chinook"
username = "larry"
# engine = create_engine(f"{dialect}+{driver}://{username}:{password}@{hostname}:{port}/{database})")

connection = psy.connect(host=hostname, database=database, user=username, password=password, port=port)
cursor = connection.cursor()

cursor.execute("SELECT table_schema, table_name FROM information_schema.tables WHERE (table_schema = 'public')")
fetched_list = cursor.fetchall()
table_list = [pair[1] for pair in fetched_list]
print(table_list)

['film', 'artist', 'album', 'employee', 'customer', 'invoice', 'invoice_line', 'track', 'playlist', 'playlist_track', 'genre', 'media_type', 'film_actor', 'film_category', 'actor', 'category']


Since using sqlalchemy has been tried and is it failed, we'll be manually importing the data into pandas dataframes, writing our own function, and using that repeatedly. We get the column names from 

In [6]:
genre, track, line, invoice = "genre_id", "track_id", "invoice_line_id", "invoice_id"
customer, unit, quant, jazz = "customer_id", "unit_price", "quantity", "Jazz"

def get_column_names(table_name : str):
    cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
    
    return [desc[0] for desc in cursor.description]


def import_from_sql(table_name : str, make_index : bool = True):
    cursor.execute(f"SELECT * FROM {table_name};")
    colnames = [desc[0] for desc in cursor.description]
    tuples = cursor.fetchall()
    dictry = {column : [] for column in colnames}

    for row in tuples:
        for i, column in enumerate(colnames):
            dictry[column].append(row[i])
    if make_index:
        return pd.DataFrame.from_dict(dictry).set_index(colnames[0])
    else:
        return pd.DataFrame.from_dict(dictry)

#Importing the dataframes as we go, only using the data we need, since we only care about Jazz, we're renaming all other genre names to Not Jazz
genre_df = import_from_sql("genre")
genre_df.loc[genre_df["name"] != jazz] = "Not Jazz"

# Converting the track ids to the genre names Jazz and Not Jazz
tracks_df = import_from_sql("track").drop(columns=[col for col in get_column_names("track") if col not in [genre, track]])
tracks_df = tracks_df.merge(genre_df, left_on=genre, right_index=True).rename(columns={"name" : "genre"}).drop(columns=[genre])
del genre_df

# Doing the same for all invoice lines and invoices, and for invoices with multiple lines; if it has one line with Jazz, it has Jazz,
inv_line_df = import_from_sql("invoice_line", False).set_index(invoice)
inv_line_df = inv_line_df.drop(columns=[col for col in get_column_names("invoice_line") if col not in [invoice, track]])
inv_line_df = inv_line_df.merge(tracks_df, left_on=track, right_index=True).drop(columns=[track]).reset_index()
inv_line_df = inv_line_df.sort_values("genre").drop_duplicates(invoice, keep="first").set_index(invoice).sort_index()
del tracks_df

# to link the invoices to customers, we need the invoice table as well, then the customer table
invoice_df = import_from_sql("invoice").drop(columns=[col for col in get_column_names("invoice") if col not in [invoice, customer]])
invoice_df = invoice_df.join(inv_line_df).reset_index().drop(columns=[invoice])
invoice_df = invoice_df.sort_values("genre").drop_duplicates(customer, keep="first").set_index(customer).sort_index()
customer_total_df = invoice_df
customer_genre_df = invoice_df
del inv_line_df

# This gives us a list of who has, and who hasn't bought Jazz music at some point
# Linking this back to invoices, we can sum the total spending of each customer, so we can check larry's hypothesis.
invoice_df = import_from_sql("invoice", False).drop(columns=[col for col in get_column_names("invoice") if col not in [invoice, customer]]).set_index(customer)
customer_total_df = customer_total_df.join(invoice_df)
inv_line_df = import_from_sql("invoice_line", False).set_index(invoice).drop(columns=[col for col in get_column_names("invoice_line") if col not in [invoice, unit, quant]])
customer_total_df = customer_total_df.merge(inv_line_df, left_on=invoice, right_index=True).drop(columns=[invoice])
customer_total_df[unit] = pd.to_numeric(customer_total_df[unit], downcast="float")

# We have all the information we need now, so we mold this a little bit to compare the two statistics:
# Average total spending of people who HAVE bought Jazz music vs people who's NEVEr bought Jazz music. 
customer_total_df[unit] = customer_total_df[unit]*customer_total_df[quant]
customer_total_df = customer_total_df.groupby(customer).sum().join(customer_genre_df)
customer_total_df = customer_total_df.set_index("genre").drop(columns=[quant]).rename(columns={unit : "total spending"})
customer_total_df = customer_total_df.groupby("genre").mean()
del invoice_df, inv_line_df, customer_genre_df
customer_total_df


,total spending
genre,
Jazz,38.964375
Not Jazz,40.064445


Clearly, this disproves Larry's hypothesis, as not only do people that buy Jazz music not spend a lot more than people who don't, they actually spend less, even if not by much.

#### Part 2: the new killer-app

Next we take a look at a new killer app Larry has thought up, where people can input a trackname, and they get a list of all albums that contain that track. So far, he's tried the following, and we aim to improve upon this:
```
SELECT album.title
FROM   track
JOIN   album ON (track.album_id = album.album_id)
WHERE  LOWER(track.name) = LOWER('Enter Sandman')
```

In [7]:
def get_albums(trackname : str):
    """trackname is the name of a track, and returns a list of album names that contain the given track"""
    sql_statement = f"""SELECT album.title
                        FROM track
                        INNER JOIN album ON (track.album_id = album.album_id)
                        WHERE LOWER(track.name) = LOWER('{trackname}')"""
    cursor.execute(sql_statement)
    albumlist = [name[0] for name in cursor.fetchall()]

    return albumlist

# This fixes Larry's function very simply by changing the 'JOIN' into an 'INNER JOIN', because it's only relevant to look at matching values in both tables

for album in get_albums("Enter Sandman"):
    print(album)

Plays Metallica By Four Cellos
Black Album
